# **SpaceX  Falcon 9 First Stage Landing Prediction**

## **Launch Sites Locations Analysis**

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

In the previous exploratory data analysis notebooks, we visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this notebook, we will be performing more interactive visual analytics using `Folium`.

## Objectives

This analysis will accomplish the following:

*   Mark all launch sites on a map
*   Mark the success/failed launches for each site on the map
*   Calculate the distances between a launch site to its proximities

After completed the above, we should be able to find some geographical patterns about launch sites.

Let's first import the required Python packages:

In [1]:
import folium
import wget
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon 

## Mark all launch sites on a map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates

First, let's try to add each site's location on a map using site's latitude and longitude coordinates

In [3]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df = pd.read_csv(spacex_csv_file)

Now, we can take a look at what are the coordinates for each site.

In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610746


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. Let's visualize those locations by pinning them on a map.

We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We can use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,

In [6]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Centre'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)    

and we should find a small yellow circle near the city of Houston and we can zoom-in to see a larger circle.

Now, let's add a circle for each launch site in data frame `launch_sites`

In [7]:
# Initialise the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# Use distinct colours to show each location
colors = ['#C70039', '#15819E', '#C87F03', '#0F9613', ] 
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for site, lat, long, colour in zip(launch_sites_df['Launch Site'].values, launch_sites_df.Lat, launch_sites_df.Long, colors):
    coordinate = [lat, long]
    # Create circle
    circle = folium.Circle(coordinate, radius=1000, color=colour, fill=True).add_child(folium.Popup(site))
    # Add marker
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:{};"><b>{}</b></div>'.format(colour, site[0:4])
            )
        )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

All of the sites are far from the Equator line with `KSC LC-39A`, `CCAFS LC-40` and `CCAFS SLC-40` equidistant from it. The earth is not a perfect sphere and gravity is less at the equator that at other locations. These gravitational differences might affect the result of the launches at each sites based on the distance from the line.

With the exception of `KSC LC-39A`, the sites are in close proximity to the coast. This might fluence the type of landing selected at each location.

## Marking the success/failed launches for each site on the map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
The data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [8]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

Let's first create a `MarkerCluster` object

In [9]:
marker_cluster  = MarkerCluster()

Now we create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value

In [10]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`

In [11]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

for index, row in spacex_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    marker = folium.Marker(
        coordinate,
        icon=folium.Icon(color='white', icon_color=row['marker_color'])
        )
    marker_cluster.add_child(marker)
    
site_map    

## Calculating the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sit

Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while exploring the map, we can easily find the coordinates of any points of interests (such as railway).

In [12]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

We can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [13]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Let's mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [14]:
# find coordinate of the closest coastline
coast_lat = 28.56293
coast_long = -80.56785
launch_site_lat = 28.562302
launch_site_long = -80.577356
# Calculate distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_long, coast_lat, coast_long)

After obtained the coordinate, let's create a `folium.Marker` to show the distance.

In [15]:
distance_marker = folium.Marker(
    [coast_lat, coast_long],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )
site_map.add_child(distance_marker)

Next, we draw a `PolyLine` between a launch site to the selected coastline point

In [16]:
site = [28.562302, -80.577356]
coast = [28.56293, -80.56785]

lines = folium.PolyLine(locations=[site, coast], weight=1)
site_map.add_child(lines)                        

Let's try plotting distances to the closest city. 

In [17]:
# Melbourne is the losest city to CCAFS LC-40, CCAFS SLC-40 and KSC LC-39A. Its coordinates are:
melbourne = [28.09985, -80.63278]
# The launch sites coordinates are:
CCAFS_LC = [28.562302, -80.577356]
CCAFS_SLC =	[28.563197,	-80.576820]
KSC_LC = [28.573255, -80.646895]

# Calculate distance CCAFS_LC to Melbourne
CCAFS_LC_to_mel = calculate_distance(CCAFS_LC[0], CCAFS_LC[1], melbourne[0], melbourne[1])
# Calculate distance CCAFS_SLC to Melbourne
CCAFS_SLC_to_mel = calculate_distance(CCAFS_SLC[0], CCAFS_SLC[1], melbourne[0], melbourne[1])
# Calculate distance KSC_LC to Melbourne
KSC_LC_to_mel = calculate_distance(KSC_LC[0], KSC_LC[1], melbourne[0], melbourne[1])

# Santa Maria is the losest city to VAFB SLC-4E. Its coordinates are:
santa_maria = [34.94899, -120.44586]
# The launch site coordinates are:
VAFB_SLC = [34.632834, -120.610746]

# Calculate distance VAFB_SLC to Santa Maria
VAFB_SLC_to_santa = calculate_distance(VAFB_SLC[0], VAFB_SLC[1], santa_maria[0], santa_maria[1])

# For comparison, distance from Houston to NASA Jackson Space Center. Its coordinates are:
houston = [29.74053, -95.32288]
# The launch site coordinates for NASA are:
NASA = [29.559684, -95.083097]

# Calculate distance VAFB_SLC to Santa Maria
NASA_to_houston = calculate_distance(NASA[0], NASA[1], houston[0], houston[1])

# Plot distances marker - Melbourne
sites = [CCAFS_LC, CCAFS_SLC, KSC_LC, VAFB_SLC, NASA]
distances = [CCAFS_LC_to_mel, CCAFS_SLC_to_mel, KSC_LC_to_mel, VAFB_SLC_to_santa, NASA_to_houston]
cities = [melbourne, melbourne, melbourne, santa_maria, houston]

for site, dist, city in zip(sites, distances, cities):
    # Plot marker
    distance_marker = folium.Marker(
    [city[0], city[1]],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(dist),
        )
    )
    # Plot line
    line = folium.PolyLine(locations=[site, city], weight=1)
    # Add marker to map
    site_map.add_child(distance_marker)
    # Add line to map
    site_map.add_child(line)


In [18]:
site_map

The closest city to a SpaceX launch site is at least 38km away (38.2km from VAFB_SLC to Santa Maria). In comparision, the distance Houston to NASA's Jackson Space Center is 30.69km. It might be a regulatory requirement that launch sites be at least 20 - 30km away from a city. 

Distance analysis allows us to answer many other questions such as:
*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?

This gives context to our analysis and might help us develop other insights about the data.